# 0. 路徑

In [4]:
import threading # 因為等等要載入大量資料所以要multi thread
from multiprocessing import Queue

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

from sklearn.linear_model import LinearRegression,Lasso,Ridge
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

from sklearn.utils import shuffle
from sklearn import metrics
from sklearn.metrics import accuracy_score

import datetime
import numpy as np
import pandas as pd
from numpy import loadtxt

import os
import argparse
from tqdm import tqdm
import matplotlib.pyplot as plt


plt.rcParams['font.family']='SimHei' #顯示中文

#特殊符號: | 

In [3]:
'''
x = pd.DataFrame({'data1':np.arange(5)})
y = pd.Series([1,0,0,1,0])

from sklearn.utils import shuffle 

def train_batch_generator(x, y, bs):
    badIndex = y[y == 1].index
    goodIndex = y[y == 0].index
    print('bad index length : ' , len(badIndex))
    print('good index length : ' , len(goodIndex))

    badCnt = 0
    goodCnt = 0
    while(True):
        batch_x = pd.DataFrame()
        batch_y = pd.Series()

        for i in range(int(bs/2)):
            if(badCnt >= len(badIndex)):
                badIndex = shuffle(badIndex)
                badCnt = 0

            batch_x = batch_x.append(x.loc[badIndex[badCnt]], ignore_index=True)
            batch_y = batch_y.append(pd.Series(y.loc[badIndex[badCnt]]), ignore_index=True)
            badCnt += 1

        for i in range(int(bs/2)):
            if(goodCnt >= len(goodIndex)):
                goodIndex = shuffle(goodIndex)
                goodCnt = 0

            batch_x = batch_x.append(x.loc[goodIndex[goodCnt]], ignore_index=True)
            batch_y = batch_y.append(pd.Series(y.loc[goodIndex[goodCnt]]), ignore_index=True)
            goodCnt += 1

        yield batch_x, batch_y
        
gen = train_batch_generator(x, y, 3)
next(gen)
'''

"\nx = pd.DataFrame({'data1':np.arange(5)})\ny = pd.Series([1,0,0,1,0])\n\nfrom sklearn.utils import shuffle \n\ndef train_batch_generator(x, y, bs):\n    badIndex = y[y == 1].index\n    goodIndex = y[y == 0].index\n    print('bad index length : ' , len(badIndex))\n    print('good index length : ' , len(goodIndex))\n\n    badCnt = 0\n    goodCnt = 0\n    while(True):\n        batch_x = pd.DataFrame()\n        batch_y = pd.Series()\n\n        for i in range(int(bs/2)):\n            if(badCnt >= len(badIndex)):\n                badIndex = shuffle(badIndex)\n                badCnt = 0\n\n            batch_x = batch_x.append(x.loc[badIndex[badCnt]], ignore_index=True)\n            batch_y = batch_y.append(pd.Series(y.loc[badIndex[badCnt]]), ignore_index=True)\n            badCnt += 1\n\n        for i in range(int(bs/2)):\n            if(goodCnt >= len(goodIndex)):\n                goodIndex = shuffle(goodIndex)\n                goodCnt = 0\n\n            batch_x = batch_x.append(x.loc[good

# 1. 載入資料

In [5]:
# 載入資料 A

train = pd.read_csv('data/training-set.csv', encoding = "utf-8", header=None)
test = pd.read_csv('data/testing-set.csv', encoding = "utf-8", header=None)

train_exc = pd.read_csv('data/exception/exception_train.txt', encoding = "utf-8", header=None)
test_exc = pd.read_csv('data/exception/exception_testing.txt', encoding = "utf-8", header=None)

train.columns=['FileID','label']
test.columns=['FileID','label']


# 確認排除的FileID在training set裡面找不到

for item in train_exc:
    print(train[train['FileID']==item])
    
for item in test_exc:
    print(test[test['FileID']==item])

Empty DataFrame
Columns: [FileID, label]
Index: []
Empty DataFrame
Columns: [FileID, label]
Index: []


In [6]:

#from multiprocessing import Queue    #使用多核心的模組 Queue
#import time

#list1=[]

#def func(i):
 #   time.sleep(np.random.randint(0,5))
  #  list1.append(i)

In [9]:
import glob
path = '/data/examples/trend/data/query_log'

def ReadFile(fp):
    return pd.read_csv(
        fp, 
        names=['FileID','CustomerID','QueryTs','ProductID'], 
        dtype={'FileID': np.str, 'CustomerID': np.str, 'QueryTs': np.int32, 'ProductID': np.str} )

all_files = glob.glob(os.path.join(path, "*.csv")) # advisable to use os.path.join as this makes concatenation OS independent
file = [ReadFile(f) for f in all_files]
raw  = pd.concat(file, ignore_index=True)

In [10]:
'''
files = [f[0:4] for f in os.listdir('/data/examples/trend/data/query_log')] # 存取每日檔案的檔案夾

raw = pd.DataFrame()
def job(num, q, lock): 

    lock.acquire()    #鎖上這個線程，在完成之前不讓其他線程干擾變數。
    global raw
    for i in range(num): # 使用 hub 時才用上真正data : len(files)
 
        unit = pd.read_csv('/data/examples/trend/data/query_log/' + str(files[i]) + '.csv', encoding = "utf-8", header=None)
        unit.columns=['FileID','CustomerID','QueryTs','ProductID']
        raw = raw.append(unit, ignore_index=True)
    
    lock.release() #解鎖這個線程，開始其他線程。
    
lock = threading.Lock()  #命名一個 Lock 物件
q = Queue() # 開一個 Queue 物件

t1 = threading.Thread(target=job, args=(len(files),q,lock)) 
                                                        #打開一個名字叫 t1 的線程物件，呼叫 job
                                                        #同時t1導入 q 跟 lock 控制線程
                                                        #Thread 裡面的 function 不可以有 return, 不然會出錯。

t1.start() #啟動 t1 線程 # t2.start() 
t1.join()  #在 t1 線程結束前阻止程式繼續運行

print(raw.shape)
raw.head()
'''

'\nfiles = [f[0:4] for f in os.listdir(\'/data/examples/trend/data/query_log\')] # 存取每日檔案的檔案夾\n\nraw = pd.DataFrame()\ndef job(num, q, lock): \n\n    lock.acquire()    #鎖上這個線程，在完成之前不讓其他線程干擾變數。\n    global raw\n    for i in range(num): # 使用 hub 時才用上真正data : len(files)\n \n        unit = pd.read_csv(\'/data/examples/trend/data/query_log/\' + str(files[i]) + \'.csv\', encoding = "utf-8", header=None)\n        unit.columns=[\'FileID\',\'CustomerID\',\'QueryTs\',\'ProductID\']\n        raw = raw.append(unit, ignore_index=True)\n    \n    lock.release() #解鎖這個線程，開始其他線程。\n    \nlock = threading.Lock()  #命名一個 Lock 物件\nq = Queue() # 開一個 Queue 物件\n\nt1 = threading.Thread(target=job, args=(len(files),q,lock)) \n                                                        #打開一個名字叫 t1 的線程物件，呼叫 job\n                                                        #同時t1導入 q 跟 lock 控制線程\n                                                        #Thread 裡面的 function 不可以有 return, 不然會出錯。\n\nt1.start() #啟動 t1 線程 # t2.star

In [11]:
print(raw.shape)
raw.head()

(83273110, 4)


,FileID,CustomerID,QueryTs,ProductID
0,63676131ae4f15699db65bd7f9a2a18a,49075aad975fa57e805c62a8494a75e4,1495497600,c105a0
1,ba04807b9b65835482302fab98fc523f,da02366776ceaae99adeed9c7140307f,1495497600,c105a0
2,16ea975c524b69ca3f2561969fabed6b,437276bc51f1603b0c8b41119019d813,1495497604,c76d58
3,1941c3fe0c5c2d53ba94d0eab99f169c,a286ad2bfd4a05d98a3a393b21180aaf,1495497612,c105a0
4,1941c3fe0c5c2d53ba94d0eab99f169c,a286ad2bfd4a05d98a3a393b21180aaf,1495497615,c105a0


In [12]:
# 載入資料 B 
'''
files = [f[0:4] for f in os.listdir('/data/examples/trend/data/query_log')] # 存取每日檔案的檔案夾

raw = pd.DataFrame()

for i in range(3): # 使用 hub 時才用上真正data : len(files)
 
    unit = pd.read_csv('/data/examples/trend/data/query_log/' + str(files[i]) + '.csv', encoding = "utf-8", header=None)
    unit.columns=['FileID','CustomerID','QueryTs','ProductID']
    raw = raw.append(unit, ignore_index=True)
'''

raw['cnt'] = 1
raw['date'] = pd.to_datetime(raw['QueryTs'], unit='s').dt.date
raw['time'] = pd.to_datetime(raw['QueryTs'], unit='s').dt.time
raw['hour'] = pd.to_datetime(raw['QueryTs'], unit='s').dt.hour
raw['week'] = pd.to_datetime(raw['QueryTs'], unit='s').dt.weekday_name

#raw.tail()

train_raw = pd.merge(raw, train, on=['FileID'])
train_raw['ProductID'] = train_raw['ProductID'].astype(str) # 因為值中有數字文字
train_raw.replace({'ProductID':{'55649': '055649'}}) # 因為raw data有少值
train_raw = train_raw.sort_values(['CustomerID','FileID','QueryTs'])

(83273110, 9) (52518, 2) (54250245, 10)
['7acab3' '055649' '634e6b' 'c76d58' 'c105a0' 'e47f04' '885fab' '26a5d0'
 'a310bb' 'dd8d4a' 'd465fc' '533133' '262880' 'b93794' '8541a0' '218578'
 '3ea8c3' '05b409' '20f8a5' '0374c4' 'cc3a6a' '8452da' 'aaa9c8' '0cdb7a'
 '3c2be6' '75f310' 'fec24f']


,FileID,CustomerID,QueryTs,ProductID,cnt,date,time,hour,week,label
6872722,f096e1c6e0cbaf10389fbf427b4d341f,0000006fa286976bf35ea17f1f19bc7a,1493364274,7acab3,1,2017-04-28,07:24:34,7,Friday,0
6873003,f096e1c6e0cbaf10389fbf427b4d341f,0000006fa286976bf35ea17f1f19bc7a,1493531993,7acab3,1,2017-04-30,05:59:53,5,Sunday,0
119508,19308434813502167aaef38f578981a4,00000145d9062eada528bace5fb4864e,1490544224,7acab3,1,2017-03-26,16:03:44,16,Sunday,0
32156382,ee6a1280be5c96d7b2461de6b7578180,00000145d9062eada528bace5fb4864e,1492708112,7acab3,1,2017-04-20,17:08:32,17,Thursday,0
32158918,ee6a1280be5c96d7b2461de6b7578180,00000145d9062eada528bace5fb4864e,1492962863,7acab3,1,2017-04-23,15:54:23,15,Sunday,0


In [15]:
print(raw.shape, train.shape, train_raw.shape)
print(train_raw.ProductID.unique())
train_raw.head()

(83273110, 9) (52518, 2) (54250245, 10)
['7acab3' '055649' '634e6b' 'c76d58' 'c105a0' 'e47f04' '885fab' '26a5d0'
 'a310bb' 'dd8d4a' 'd465fc' '533133' '262880' 'b93794' '8541a0' '218578'
 '3ea8c3' '05b409' '20f8a5' '0374c4' 'cc3a6a' '8452da' 'aaa9c8' '0cdb7a'
 '3c2be6' '75f310' 'fec24f']


,FileID,CustomerID,QueryTs,ProductID,cnt,date,time,hour,week,label
6872722,f096e1c6e0cbaf10389fbf427b4d341f,0000006fa286976bf35ea17f1f19bc7a,1493364274,7acab3,1,2017-04-28,07:24:34,7,Friday,0
6873003,f096e1c6e0cbaf10389fbf427b4d341f,0000006fa286976bf35ea17f1f19bc7a,1493531993,7acab3,1,2017-04-30,05:59:53,5,Sunday,0
119508,19308434813502167aaef38f578981a4,00000145d9062eada528bace5fb4864e,1490544224,7acab3,1,2017-03-26,16:03:44,16,Sunday,0
32156382,ee6a1280be5c96d7b2461de6b7578180,00000145d9062eada528bace5fb4864e,1492708112,7acab3,1,2017-04-20,17:08:32,17,Thursday,0
32158918,ee6a1280be5c96d7b2461de6b7578180,00000145d9062eada528bace5fb4864e,1492962863,7acab3,1,2017-04-23,15:54:23,15,Sunday,0


In [ ]:
train_raw.to_csv('train_raw.csv')

In [13]:
'''
observe1 = train_raw.groupby(['FileID','CustomerID','ProductID'])[['cnt','label']].sum()
observe1 = observe1.sort_values('label', ascending=0)
aa = observe1.loc[observe1['label']==1]
aa.head()
'''

"\nobserve1 = train_raw.groupby(['FileID','CustomerID','ProductID'])[['cnt','label']].sum()\nobserve1 = observe1.sort_values('label', ascending=0)\naa = observe1.loc[observe1['label']==1]\naa.head()\n"

In [14]:
'''
observe2 = train_raw.groupby(['week'])[['cnt','label']].sum() 
observe2['fr_rate'] = observe2['label'] / observe2['cnt']
observe2 = observe2.sort_values('fr_rate', ascending=0)
print(observe2)

observe3 = train_raw.groupby(['ProductID'])[['cnt','label']].sum() 
observe3['fr_rate'] = observe3['label'] / observe3['cnt']
observe3 = observe3.sort_values('fr_rate', ascending=0)
print(observe3)
'''

"\nobserve2 = train_raw.groupby(['week'])[['cnt','label']].sum() \nobserve2['fr_rate'] = observe2['label'] / observe2['cnt']\nobserve2 = observe2.sort_values('fr_rate', ascending=0)\nprint(observe2)\n\nobserve3 = train_raw.groupby(['ProductID'])[['cnt','label']].sum() \nobserve3['fr_rate'] = observe3['label'] / observe3['cnt']\nobserve3 = observe3.sort_values('fr_rate', ascending=0)\nprint(observe3)\n"

# 2A. Feature Engineering

In [ ]:
# aggregate

# 單日File被使用幾次
DayFil    = train_raw.groupby(['FileID', 'date']).size()
DayFilMax = DayFil.groupby(level=0).max()
DayFilMin = DayFil.groupby(level=0).min()
DayFilMea = DayFil.groupby(level=0).mean()


DayCs  = train_raw.groupby(['FileID', 'date', 'CustomerID']).size()
DayPr  = train_raw.groupby(['FileID', 'date', 'ProductID']).size()
Pr     = train_raw.groupby(['FileID', 'ProductID']).size()
Cs     = train_raw.groupby(['FileID', 'CustomerID']).size()
Day    = train_raw.groupby(['FileID', 'date']).size()

# 單日File被多少客人使用
DayTCs    = DayCs.groupby(level=(0,1)).size()
DayTCsMax = DayTCs.groupby(level=0).max()
DayTCsMin = DayTCs.groupby(level=0).min()
DayTCsMea = DayTCs.groupby(level=0).mean()

# 單日File被同一客人使用幾次
DaySCsMax = DayCs.groupby(level=0).max()
DaySCsMin = DayCs.groupby(level=0).min()
DaySCsMea = DayCs.groupby(level=0).mean()

# 單日File被多少產品使用
DayTPr    = DayPr.groupby(level=(0,1)).size()
DayTPrMax = DayTPr.groupby(level=0).max()
DayTPrMin = DayTPr.groupby(level=0).min()
DayTPrMea = DayTPr.groupby(level=0).mean()

# 單日File被同一產品使用幾次
DaySPrMax = DayPr.groupby(level=0).max()
DaySPrMin = DayPr.groupby(level=0).min()
DaySPrMea = DayPr.groupby(level=0).mean()


TCs    = Cs.groupby(level=(0)).size() # 總共File被多少客人使用
TPr    = Pr.groupby(level=(0)).size() # 總共File被多少產品使用
Day    = Day.groupby(level=(0)).size() # 總共File被使用幾天
Fil    = train_raw.groupby(['FileID']).size() # 總共File被使用幾次

train_agg = pd.concat([DayFilMax, DayFilMin, DayFilMea,
                       DayTCsMax, DayTCsMin, DayTCsMea,
                       DaySCsMax, DaySCsMin, DaySCsMea,
                       DayTPrMax, DayTPrMin, DayTPrMea,
                       DaySPrMax, DaySPrMin, DaySPrMea,
                       TCs,    TPr,   Day,    Fil,     ], axis=1)

train_agg.columns = ['DayFilMax', 'DayFilMin', 'DayFilMea',
                     'DayTCsMax', 'DayTCsMin', 'DayTCsMea',
                     'DaySCsMax', 'DaySCsMin', 'DaySCsMea',
                     'DayTPrMax', 'DayTPrMin', 'DayTPrMea',
                     'DaySPrMax', 'DaySPrMin', 'DaySPrMea',
                     'TCs',   'TPr',  'Day',  'Fil'      ]

train_agg = pd.DataFrame(train_agg).reset_index()
train_agg.head()

In [ ]:
# one-hot encoding

Pr = train_raw[['FileID', 'ProductID']]
Hr = train_raw[['FileID', 'hour']]
Wk = train_raw[['FileID', 'week']]

train_prd = pd.concat([Pr, pd.get_dummies(Pr.ProductID)], 1).groupby(['FileID']).sum().reset_index()
train_hur = pd.concat([Hr, pd.get_dummies(Hr.hour)], 1).groupby(['FileID']).sum().reset_index()
train_wek = pd.concat([Wk, pd.get_dummies(Wk.week)], 1).groupby(['FileID']).sum().reset_index()

train_dum = pd.merge(train_prd, train_hur, on='FileID')
train_dum = pd.merge(train_dum, train_wek, on='FileID')

train_dum.head()

In [ ]:
# 使用間隔

def regular(x):
    return ( x%3600 <= 10 ) | ( x%3600 >= 3590 )

#print(regular(np.array([1,3599,3601,450,720,4150])))

def outlier(x):
    Q1 = np.percentile(x, 25)
    Q3 = np.percentile(x, 75)
    h = Q3 - Q1
    return ((Q3 + 0.5*h) >= x) & (x >= (Q1 - 0.5*h))

#print(outlier(np.array([1,100,100,100,100,100,100])))

train_freq = train_raw[['CustomerID','FileID','QueryTs','label']] # .head(9000)

train_freq['Delta1']    = train_freq.groupby(['FileID'])['QueryTs'].transform(lambda x: x-x.shift(1))
train_freq['Delta1_d']  = train_freq.groupby(['FileID'])['Delta1'].transform(lambda x: abs(x-x.shift(1)))
#train_freq['Delta2']    = train_freq.groupby(['CustomerID','FileID'])['QueryTs'].transform(lambda x: x-x.shift(1))
#train_freq['Delta2_d']  = train_freq.groupby(['CustomerID','FileID'])['Delta2'].transform(lambda x: abs(x-x.shift(1)))

# 測: 是否為3600秒一單位)

train_freq1 = train_freq.drop(['Delta1_d'], axis=1).dropna() #.loc[train_freq['label']==0]
train_freq1['Regular'] = train_freq1['Delta1'].apply(regular)
train_freq1mean = train_freq1.groupby(['FileID'])['Regular'].mean().reset_index()
train_freq1max  = train_freq1.groupby(['FileID'])['Regular'].max().reset_index()

# 測: 規律性 (標準差)

train_freq2 = train_freq.dropna() # .loc[train_freq['label']==0]
train_freq2['Delta_OL'] = train_freq2.groupby(['CustomerID','FileID'])['Delta1_d'].transform(outlier)
train_freq2 = train_freq2[train_freq2['Delta_OL'] != 0]
train_freq2['Delta_sd'] = train_freq2.groupby(['CustomerID','FileID'])['Delta1_d'].transform(lambda x: np.std(x)/np.mean(x))
train_freq2mean = train_freq2.groupby(['FileID'])['Delta_sd'].mean().fillna(0).reset_index()
train_freq2max = train_freq2.groupby(['FileID'])['Delta_sd'].max().fillna(0).reset_index()


In [ ]:
train_freq1.head()

In [ ]:
train_freq2.head()

In [ ]:
print(train_freq1.shape, train_freq2.shape, train_dum.shape, train_agg.shape)


# 2B. 資料清整

In [ ]:
# 全部合併

train_all = pd.merge(train_freq1mean, train_freq1max, on=['FileID'], how='outer')
train_all = pd.merge(train_all, train_freq2mean, on=['FileID'], how='outer')
train_all = pd.merge(train_all, train_freq2max, on=['FileID'], how='outer')
train_all = pd.merge(train_all, train_dum, on=['FileID'], how='outer')
train_all = pd.merge(train_all, train_agg, on=['FileID'], how='outer')

train_all.head()

# index & column 相merge時: train_all = pd.merge(train_all, train_agg, how='left', left_on=['FileID'], right_index=True)

In [ ]:
print(train_all.columns)
train_all.to_csv("train_all.csv")

In [ ]:
# 去除 outlier

In [ ]:
# 補值 (interval: 同一file同一customer之間的時間差)

train_all[['FMin']]     = train_all[['FMin']].fillna(value=0) # 沒有interval
train_all[['Delta_sd']] = train_all[['Delta_sd']].fillna(value=100) # 沒有兩個以上interval

check1 = len(train_all[train_all['FMin'].isnull()])
check2 = len(train_all[train_all['Delta_sd'].isnull()])
check3 = len(train_all[train_all['20f8a5'].isnull()])
check4 = len(train_all[train_all['DayTCsMea'].isnull()])

print(check1)
print(check2)
print(check3)
print(check4)


In [ ]:
# Normalization
'''
def normalize(x, axis, method, minmax_range =(0,1)):
    if method == 'z-score':
        scale_a = preprocessing.scale(a, axis=axis)
    elif method== 'minmax':    
        scale_a = preprocessing.minmax_scale(a, axis=axis, feature_range=minmax_range) #default feature range 0~1
    return scale_a
axis =0
scale_a1 = normalize(a, axis, method = 'z-score')
scale_a2 = normalize(a, axis, method = 'minmax', minmax_range=(0,1))
print(scale_a1)
'''

# 3. 探索性資料分析

In [ ]:
# 散布圖
'''
color = "rbg"
color = [color[y[i]] for i in range(len(y))]

plt.subplot(121)
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.title('Actual')
plt.xlabel('X1')
plt.ylabel('X2')

plt.subplot(122)
plt.scatter(X[:, 0], X[:, 1], c=prediction)
plt.title('Prediction')
plt.xlabel('X1')
plt.ylabel('X2')
plt.show()
'''

In [ ]:
# 長條圖

# 4. 建立模型

In [ ]:
# 迭代 train model - 避免imbalance (test data不用拆，直接看AUC&ROC分數即可)

def train_batch_generator(x, y, bs):
    badIndex = y[y == 1].index
    goodIndex = y[y == 0].index

    while(True):
        newBad_ind = shuffle(badIndex)
        newgood_ind = shuffle(goodIndex)

        newBad_ind = newBad_ind[:int(bs/2)]
        newgood_ind = newgood_ind[:int(bs/2)]

        batch_x = x.loc[newBad_ind]
        batch_y = y.loc[newBad_ind]

        batch_x = batch_x.append(x.loc[newgood_ind], ignore_index=True)
        batch_y = batch_y.append(y.loc[newgood_ind], ignore_index=True)

        yield batch_x, batch_y


In [ ]:
# 分成 train & test

train_lbl = pd.merge(train_all, train, on=['FileID'], how='left')

train_all = train_all.sort_values(['FileID'])
train_lbl = train_lbl.sort_values(['FileID'])

X = train_all.drop(['FileID','533133','218578','262880'], axis=1)
y = train_lbl['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state= 68, shuffle=False)

gen = train_batch_generator(X_train, y_train, 10000)


In [1]:
model = XGBClassifier()
    
model.fit(X_train, y_train, 
          eval_set=[(X_train, y_train), (X_test, y_test)],
          eval_metric='auc',
          verbose=False)

NameError: name 'XGBClassifier' is not defined

In [ ]:
y_modl = model.predict(bat_x) # 找 bat_x 的auc score
y_pred = model.predict(X_test)# 找 X_test 的auc score

#y_pred = model.predict_proba(X_test)[:,1] # 找 X_test 的auc score

auc1    = metrics.roc_auc_score(bat_y, y_modl)
auc2    = metrics.roc_auc_score(y_test, y_pred)

print(auc1, auc2)

In [ ]:
'''
bat_x, bat_y = next(gen)
print(bat_x, bat_y)
'''

In [ ]:
# LASSO / Ridge

model = Lasso(alpha=1, normalize=False, copy_X=True, fit_intercept=True, max_iter=1000)
model.fit(X_train, y_train)
prediction = model.predict(X)
print('accuracy:')
print(model.score(X_test, y_test))

model = Ridge(alpha=1, normalize=False, copy_X=True, fit_intercept=True, max_iter=1000)
model.fit(X_train, y_train)
prediction = model.predict(X)
print('accuracy:')
print(model.score(X_test, y_test))

In [ ]:
# SVM

model = SVC(kernel='poly',coef0=0, degree=3)
model.fit(X_train, y_train)
#print('accuracy:', model.accuracy_score(X_test, y_test))


In [76]:
model = XGBClassifier()

for i in range(10):
    bat_x, bat_y = next(gen)
    
    model.fit(bat_x, bat_y, 
              eval_set=[(bat_x, bat_y), (X_test, y_test)],
              eval_metric='auc',
              verbose=False)
    
    y_modl = model.predict(bat_x) # 找 bat_x 的auc score
    y_pred = model.predict(X_test)# 找 X_test 的auc score
    
    #y_pred = model.predict_proba(X_test)[:,1] # 找 X_test 的auc score
    
    auc1    = metrics.roc_auc_score(bat_y, y_modl)
    auc2    = metrics.roc_auc_score(y_test, y_pred)
    
    print(auc1, auc2)
 

0.516508391608 0.498755185335
0.515652797203 0.503177201456
0.51476013986 0.501870563781
0.516071328671 0.498903059292
0.517938111888 0.498438829752
0.51476013986 0.501891171451
0.512574825175 0.500099696566
0.512574825175 0.499773037147
0.514323076923 0.501791474885
0.516882517483 0.497242192478


"\nfig, ax = pyplot.subplots()\nax.plot(x_axis, results['validation_0']['logloss'], label='Train')\nax.plot(x_axis, results['validation_1']['logloss'], label='Test')\nax.legend()\npyplot.ylabel('Log Loss')\npyplot.title('XGBoost Log Loss')\npyplot.show()\n"

In [ ]:
'''
eval_set = [(X_test, y_test)]
#print(model.score(X_test, y_test))

y_pred = model.predict(X_test)
print metrics.roc_auc_score(y_test,y_pred)
auc_score = roc_auc_score(y_test, pipe.predict_proba(X_test)[:,1]))

if auc_score < 0.5:
    fpr_svc, tpr_svc, _ = roc_curve(y_test, pipe.predict_proba(X_test)[:,1], pos_label=0)
    auc_score = 1 - auc_score
else:
    fpr_svc, tpr_svc, _ = roc_curve(y_test, pipe.predict_proba(X_test)[:,1])

plt.plot(fpr_svc, tpr_svc, label='ROC Curve', color='cyan')
plt.plot([0,1], [0,1], color='black', linestyle='--')
plt.xlim([0,1])
plt.ylim([0,1])
plt.show()
'''

'''
fig, ax = pyplot.subplots()
ax.plot(x_axis, results['validation_0']['logloss'], label='Train')
ax.plot(x_axis, results['validation_1']['logloss'], label='Test')
ax.legend()
pyplot.ylabel('Log Loss')
pyplot.title('XGBoost Log Loss')
pyplot.show()
'''

In [ ]:
# XGBoost

# eval_metric 選項: rmse, logloss, error, auc, merror, mlogloss or custom define

# model.predict_proba will return the probility
# model.predict will return the predict label (use 0.5 as threshold)
y_pred = model.predict_proba(X_test)
y_pred[:5]

# we use model.predict to get the label
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred )
print("Accuracy:" % (accuracy * 100.0))

# we can show the feature importances for our features
print(model.feature_importances_)
# import the plot_importance function to visualize the feature importance
from xgboost import plot_importance
plot_importance(model)
plt.show()

from xgboost import plot_tree
from matplotlib.pylab import rcParams

plot_tree(model, num_trees=1)
# plt.title("max_depth = 100, with gamma = 10")
# plt.savefig("tree_with_max_depth_gamma", dpi = 700)
'''

In [ ]:
# 5. 組合模型 (Ensemble)

In [ ]:
# Stacking

In [ ]:
# 6. 上傳結果

NameError: name 'train' is not defined